In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
from lpne.models import DcsfaNmf

MODEL_FILE = "/hpc/home/mk423/Anxiety/FullDataWork/Models/Final_mt_Model_500_epochs.pt"
DATA_PATH = "/work/mk423/Anxiety/"
PROJECT_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Projections/"
FIGURE_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Figures/"
data_file = DATA_PATH + "EPM_test_dict_May_17.pkl"
train_file = DATA_PATH + "EPM_train_dict_May_17.pkl"
val_file = DATA_PATH + "EPM_val_dict_May_17.pkl"
proj_file = PROJECT_PATH + "EPM_Holdout_Projections.csv"
mean_file = PROJECT_PATH + "EPM_Holdout_mean_scores.csv"

model = torch.load(MODEL_FILE,map_location="cpu")
model.device="cpu"


old_feature_list = ["X_power_1_2","X_coh_1_2","X_gc_1_2"]
feature_weights = [10,1,1]



import os, sys
umc_data_tools_path = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
sys.path.append(umc_data_tools_path)
import umc_data_tools as umc_dt

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc/home/mk423/.local/lib/python3.7/site-packages/lpne/pipelines/__init__.py:14: UserWarning: Could not load lpne/pipelines/default_params.yaml!
  warnings.warn("Could not load lpne/pipelines/default_params.yaml!")


In [2]:
dataDict = pickle.load(open(data_file,"rb"))
np.unique(dataDict["y_mouse"])

array(['Mouse0641', 'Mouse39115', 'Mouse39121', 'Mouse39122',
       'Mouse39132', 'Mouse39135', 'Mouse6674', 'Mouse69061',
       'Mouse69071', 'Mouse69075', 'Mouse8893'], dtype='<U10')

In [3]:
dataDict = pickle.load(open(val_file,"rb"))
np.unique(dataDict["y_mouse"])

array(['Mouse04193', 'Mouse0633', 'Mouse0642', 'Mouse39125', 'Mouse69065'],
      dtype='<U10')

In [8]:
data_dictionaries = [train_file,val_file,data_file]
data_type = ["train","val","test"]

mouse_names = []
split = []
open_arm_perc = []
closed_arm_perc = []
center_perc = []
time_in_task_list = []

for file, group in zip(data_dictionaries,data_type):
    dataDict = pickle.load(open(file,"rb"))
    
    for mouse in np.unique(dataDict["y_mouse"]):
        
        #Isolate the in task windows
        mouse_mask = dataDict["y_mouse"]==mouse
        nan_mask = dataDict["y_ROI"] > 0
        in_task_mask = 1-dataDict["y_Homecage"]
        
        mask = np.logical_and(mouse_mask,nan_mask)
        mask = np.logical_and(mask,in_task_mask)

        mouse_in_task_mask = np.logical_and(mouse_mask,in_task_mask)
        print(mouse,np.sum(mouse_in_task_mask))
        
        mouse_time = dataDict["y_time"][mask==1]
        task_start_time = np.min(mouse_time)
        task_end_time = task_start_time + 300
        time_mask = dataDict["y_time"] < task_end_time
        print(mouse,task_start_time,task_end_time)
        mask = np.logical_and(mask,time_mask)
        total_time_mask = np.logical_and(mouse_mask,in_task_mask)
        total_time_mask = np.logical_and(total_time_mask,time_mask)
        
        #Baseline Info
        time_in_task = np.sum(total_time_mask)#np.max(dataDict["y_time"][mouse_in_task_mask==1]) - np.min(dataDict["y_time"][mouse_in_task_mask==1])
        
        ca_mask = np.logical_or(dataDict["y_ROI"][mask==1]==1,
                                dataDict["y_ROI"][mask==1]==3)
        
        oa_mask = np.logical_or(dataDict["y_ROI"][mask==1]==2,
                                dataDict["y_ROI"][mask==1]==4)
        
        c_mask = dataDict["y_ROI"][mask==1]==5
        
        #frequency
        n_transitions_to_oa = np.sum(ca_mask[1:] != ca_mask[:-1])
        
        #time in task
        perc_ca = np.sum(ca_mask) / time_in_task
        perc_oa = np.sum(oa_mask) / time_in_task
        perc_c = np.sum(c_mask) / time_in_task
        
        #distance
        #velocity_nan_mask = dataDict["y_vel"] >=0
        #vel_mask = np.logical_and(mask,velocity_nan_mask)
        #distance = np.sum(dataDict["y_vel"][vel_mask==1])
        
        mouse_names.append(mouse)
        split.append(group)
        open_arm_perc.append(perc_oa)
        closed_arm_perc.append(perc_ca)
        center_perc.append(perc_c)
        time_in_task_list.append(time_in_task)
    

Mouse04201 314
Mouse04201 343 643
Mouse04202 326
Mouse04202 348 648
Mouse04205 9
Mouse04205 321 621
Mouse04215 352
Mouse04215 327 627
Mouse0630 603
Mouse0630 162 462
Mouse0634 570
Mouse0634 137 437
Mouse0643 169
Mouse0643 164 464
Mouse1551 653
Mouse1551 148 448
Mouse39114 198
Mouse39114 341 641
Mouse39124 337
Mouse39124 328 628
Mouse39133 368
Mouse39133 420 720
Mouse6291 618
Mouse6291 209 509
Mouse6292 629
Mouse6292 220 520
Mouse6293 620
Mouse6293 214 514
Mouse69064 292
Mouse69064 325 625
Mouse69074 332
Mouse69074 326 626
Mouse8580 549
Mouse8580 180 480
Mouse8581 441
Mouse8581 147 447
Mouse8582 42
Mouse8582 159 459
Mouse8891 602
Mouse8891 149 449
Mouse8894 372
Mouse8894 143 443
Mouse04193 289
Mouse04193 353 653
Mouse0633 611
Mouse0633 139 439
Mouse0642 581
Mouse0642 144 444
Mouse39125 284
Mouse39125 319 619
Mouse69065 380
Mouse69065 366 666
Mouse0641 264
Mouse0641 147 447
Mouse39115 343
Mouse39115 324 624
Mouse39121 346
Mouse39121 339 639
Mouse39122 328
Mouse39122 321 621
Mouse39132 29

In [9]:
df = pd.DataFrame({"mouse":mouse_names,
                   "split":split,
                   "time-in-task":time_in_task_list,
                   "OpenArmPerc":open_arm_perc,
                   "ClosedArmPerc":closed_arm_perc,
                   "CenterPerc":center_perc,
                  })

df.to_csv(PROJECT_PATH + "EPM_perc_time_behaviors_5_mins.csv",index=False)

In [10]:
df

,mouse,split,time-in-task,OpenArmPerc,ClosedArmPerc,CenterPerc
0,Mouse04201,train,283,0.137809,0.770318,0.091873
1,Mouse04202,train,300,0.100000,0.740000,0.160000
2,Mouse04205,train,9,0.222222,0.777778,0.000000
3,Mouse04215,train,295,0.067797,0.796610,0.135593
4,Mouse0630,train,296,0.158784,0.682432,0.158784
5,Mouse0634,train,274,0.233577,0.598540,0.167883
6,Mouse0643,train,169,0.213018,0.692308,0.094675
7,Mouse1551,train,292,0.297945,0.582192,0.119863
8,Mouse39114,train,169,0.100592,0.591716,0.301775
9,Mouse39124,train,296,0.175676,0.594595,0.229730


In [8]:
mouse_mask

array([False, False, False, ...,  True,  True,  True])

In [9]:
dataDict.keys()

dict_keys(['X_psd', 'X_ds', 'X_power_1_2', 'X_coh_1_2', 'X_gc_1_2', 'y_mouse', 'y_locx', 'y_locy', 'y_ROI', 'y_vel', 'y_expDate', 'y_Homecage', 'y_time', 'mice', 'info'])